In [1]:
import pandas as pd
import os
import re
import json

def create_department_csv(department_code):
    """
    Creates a CSV file containing all communes for a given department.
    
    Args:
        department_code (str): The department code (e.g., '06', '75', '2A')
    
    Returns:
        str: The path of the created file
    """
    # Read source file
    # Specify dtypes for code columns to preserve leading zeros and avoid .0 suffix
    dtype_dict = {
        'code_insee': str,
        'dep_code': str,
        'code_postal': str,
        'codes_postaux': str,
        'reg_code': str,
        'canton_code': str,
        'epci_code': str
    }
    
    df = pd.read_csv('communes-france-2025.csv', dtype=dtype_dict, low_memory=False)
    
    # Filter by department
    df_dept = df[df['dep_code'] == str(department_code)]
    
    # Create directory if not exists
    os.makedirs('departements', exist_ok=True)
    
    # Create filename
    output_file = f'departements/communes-departement{department_code}.csv'
    
    # Save file
    df_dept.to_csv(output_file, index=False)
    
    print(f"CSV created: {output_file} ({len(df_dept)} communes)")
    
    return output_file


def create_department_json(department_code):
    """
    Creates a JSON file containing all gas stations for a given department.
    
    Args:
        department_code (str): The department code (e.g., '06', '75', '2A')
    
    Returns:
        str: The path of the created file
    """
    # Read source JSON file
    with open('prix-carburants-quotidien-preprocessed.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Filter stations by department code
    filtered_data = [station for station in data if station.get('dep_code') == str(department_code)]
    
    # Create directory if not exists
    os.makedirs('departements', exist_ok=True)
    
    # Create filename
    output_file = f'departements/stations-departement{department_code}.json'
    
    # Save file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_data, f, ensure_ascii=False)
    
    print(f"JSON created: {output_file} ({len(filtered_data)} stations)")
    
    return output_file


def create_department_files(department_code):
    """
    Creates both CSV and JSON files for a given department.
    
    Args:
        department_code (str): The department code (e.g., '06', '75', '2A')
    
    Returns:
        tuple: (csv_file_path, json_file_path)
    """
    csv_file = create_department_csv(department_code)
    json_file = create_department_json(department_code)
    return csv_file, json_file

In [2]:
import subprocess
import sys
import os
import re

def run_mapping(department_code):
    """
    Runs the RML mapping for a specific department.
    
    Args:
        department_code (str): The department code.
    """
    # 1. Create the department CSV and JSON files
    csv_file_path, json_file_path = create_department_files(department_code)
    
    # 2. Update the mapping file
    mapping_file = 'mapping-optimized.ttl'
    
    # Ensure mappings directory exists
    os.makedirs('mappings', exist_ok=True)
    temp_mapping_file = f'mappings/mapping_{department_code}.ttl'
    
    with open(mapping_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Replace CSV source file (communes)
    new_content = re.sub(
        r'rml:source\s+"[^"]+\.csv"\s*;',
        f'rml:source "{csv_file_path}" ;',
        content
    )
    
    # Replace JSON source file (stations)
    new_content = re.sub(
        r'rml:source\s+"[^"]+\.json"\s*;',
        f'rml:source "{json_file_path}" ;',
        new_content
    )
    
    with open(temp_mapping_file, 'w', encoding='utf-8') as f:
        f.write(new_content)
        
    print(f"Mapping created: {temp_mapping_file}")
    
    # 3. Run the Docker command
    # Ensure outputs directory exists
    os.makedirs('outputs', exist_ok=True)
    output_file = f'outputs/output_{department_code}.ttl'
    
    print(f"Running RML mapper for department {department_code}...")
    
    # Get absolute path for volume mount
    current_dir = os.getcwd()
    
    # Docker command
    cmd = [
        "docker", "run", "--rm",
        "-v", f"{current_dir}:/data",
        "-w", "/data",
        "rmlio/rmlmapper-java:latest",
        "--mappingfile", temp_mapping_file,
        "--outputfile", output_file,
        "--serialization", "turtle"
    ]
    
    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            encoding='utf-8'
        )
        
        return_code = result.returncode
        
        if return_code == 0:
            if os.path.exists(output_file):
                print(f"Mapping completed: {output_file}")
            else:
                print("Error: Docker command succeeded but output file was not created.")
        else:
            print(f"Docker execution failed with return code: {return_code}")
            print("Error output:")
            print(result.stderr)
            
    except Exception as e:
        print(f"An error occurred during execution: {e}")

In [3]:
import time
import os
from tqdm import tqdm

# Load departments list
df_communes = pd.read_csv('communes-france-2025.csv')
# Get unique department codes and sort them
departments = sorted(df_communes['dep_code'].astype(str).unique().tolist())

# Dictionary to store results: dep_code -> (time_taken, line_count)
results = {}

# Select a subset for demonstration (e.g., first 5) to avoid long execution time
# Uncomment the next line to run for all departments
# target_departments = departments 
target_departments = departments

print(f"Running benchmark for {len(target_departments)} departments...")

for dep in tqdm(target_departments):
    
    start_time = time.perf_counter()
    
    # Run the mapping
    run_mapping(dep)
    
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    
    # Count lines in output file
    output_file = f'outputs/output_{dep}.ttl'
    line_count = 0
    if os.path.exists(output_file):
        try:
            with open(output_file, 'r', encoding='utf-8') as f:
                line_count = sum(1 for _ in f)
        except Exception as e:
            print(f"Error counting lines: {e}")
    
    results[dep] = (execution_time, line_count)
    print(f"Department {dep}: {execution_time:.4f}s, {line_count} lines")

print("\nBenchmark completed.")

FileNotFoundError: [Errno 2] No such file or directory: 'communes-france-2025.csv'

In [ ]:
# Create a DataFrame from the results
data = []
for dep, (exec_time, lines) in results.items():
    data.append({
        'Department': dep,
        'Time (s)': exec_time,
        'Lines': lines
    })

df_results = pd.DataFrame(data)

# Sort by Time
df_results_sorted = df_results.sort_values(by='Time (s)', ascending=False)

# Display the sorted table
print("Results sorted by execution time:")
try:
    display(df_results_sorted)
except NameError:
    print(df_results_sorted)

In [ ]:
from pathlib import Path
import os
from rdflib import Graph, Namespace

def merge_ttl_files_with_dedup(output_dir='outputs', merged_file='output_merged.ttl'):
    """
    Merge all TTL files from output directory into one file with automatic deduplication.
    Uses RDFLib to parse and merge graphs, which automatically handles duplicate triples.
    """
    # Get all .ttl files in the outputs directory
    output_path = Path(output_dir)
    ttl_files = sorted(output_path.glob('output_*.ttl'))
    
    if not ttl_files:
        print("No output files found to merge.")
        return
    
    print(f"Found {len(ttl_files)} files to merge...")
    
    # Create a new graph for merging
    merged_graph = Graph()
    
    # Load and merge all files
    for ttl_file in ttl_files:
        print(f"Loading {ttl_file.name}...")
        try:
            g = Graph()
            g.parse(ttl_file, format='turtle')
            print(f"  → {len(g):,} triples")
            
            # Merge into main graph (automatically deduplicates)
            merged_graph += g
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal unique triples after merge: {len(merged_graph):,}")
    
    # Bind custom namespaces
    merged_graph.bind("schema", Namespace("http://schema.org/"))
    merged_graph.bind("ex", Namespace("http://example.org/vocab#"))
    merged_graph.bind("dbo", Namespace("http://dbpedia.org/ontology/"))
    
    # Serialize the merged graph
    print(f"Writing merged file to {merged_file}...")
    merged_graph.serialize(destination=merged_file, format='turtle')
    
    # Stats
    file_size = os.path.getsize(merged_file)
    print(f"\n✓ Merge completed!")
    print(f"Output file: {merged_file}")
    print(f"Unique triples: {len(merged_graph):,}")
    print(f"File size: {file_size:,} bytes ({file_size / 1024 / 1024:.2f} MB)")

# Execute merge with deduplication
merge_ttl_files_with_dedup()


✓ Merge completed!
Output file: output_merged.ttl
Unique triples: 507,359
File size: 20,918,405 bytes (19.95 MB)


## Déduplication d'un fichier TTL existant

Si vous avez déjà un fichier merged avec des doublons, utilisez la cellule suivante pour le nettoyer.

In [6]:
from rdflib import Graph
from pathlib import Path

input_file = 'output_merged.ttl'
output_file = 'output_merged_deduplicated.ttl'

In [7]:
input_path = Path(input_file)
input_size = input_path.stat().st_size / (1024 * 1024)  # MB
g = Graph()
g.parse(input_file, format='turtle')

<Graph identifier=Ne8130bf2845c41f1850c4b025022bfa7 (<class 'rdflib.graph.Graph'>)>

In [ ]:
g.serialize(destination=output_file, format='turtle')
output_path = Path(output_file)
output_size = output_path.stat().st_size / (1024 * 1024)  # MB
reduction = ((input_size - output_size) / input_size) * 100
